# Ball Bessel Eigenproblem

\begin{align}
\nabla^2 f + \kappa^2 f &= 0 \\
f(r=1) &= 0
\end{align}

In [ ]:
import dedalus_sphere.zernike as Zernike

import numpy as np
import scipy as sp
import scipy.sparse as sparse
from scipy.special import spherical_jn
from scipy.linalg import eig
import matplotlib.pyplot as plt
%matplotlib notebook

np.set_printoptions(precision=3,suppress=True)

dim = 3
D = Zernike.operator(dim, 'D')
E = Zernike.operator(dim, 'E')


### Bessel Parameters

In [ ]:
# Shared Bessel parameters
Nmax, alpha, ell = 512, 0, 30

In [ ]:
def eigsort(A, B, cutoff=np.inf):
    vals, vecs = eig(A, b=B)
    bad = (np.abs(vals) > cutoff)
    vals[bad] = np.nan
    vecs = vecs[:,np.isfinite(vals)]
    vals = vals[np.isfinite(vals)]

    i = np.argsort(vals.real)
    vals, vecs = vals[i], vecs[:, i]

    return vals, vecs


### Analytic Solution

In [ ]:
def dispersion_zeros(ell,n,a=0,guess=None,imax=20,nk=10,eps=0.1):
    j = spherical_jn
    def F(k,deriv=False): 
        return j(ell,k,derivative=deriv) - a*j(ell+2,k,derivative=deriv)
    
    if guess == None:    
        kmax = np.pi*(n+ell/2 + eps)
        k = np.linspace(0,kmax,int(kmax*nk))
        S = np.sign(F(k))
        i = np.where(np.abs(np.roll(S,-1)-S)==2)[0]
        k = 0.5*(k[i]+k[i+1])
    else: k = guess
    
    for i in range(imax):
        dk =  F(k)/F(k,deriv=True)
        k -= dk
    
    return k

# Bessel equation, analytic eigenvalues
evalues_analytic = dispersion_zeros(ell,Nmax)


### Tau Method

In [ ]:
# Bessel equation, tau
L = D(-1) @ D(+1)
M = -E(+1)**2

L = L(Nmax, alpha, ell)
M = M(Nmax, alpha, ell).tolil()

bc = Zernike.polynomials(dim,Nmax,alpha,ell,+1)
L = sparse.vstack([L, bc])
M[-1,:] = 0

evalues_tau, evectors_tau = eigsort(L.todense(), M.todense())
evalues_tau = np.sqrt(evalues_tau.real)

### Galerkin Method

In [ ]:
# Bessel equation, galerkin
L = D(-1) @ D(+1) @ E(-1)
M = -E(+1)**2 @ E(-1)

L = L(Nmax, alpha+1, ell)
M = M(Nmax, alpha+1, ell)
M = M[:Nmax,:]

evalues_galerkin, evectors_galerkin = eigsort(L.todense(), M.todense())
evalues_galerkin = np.sqrt(evalues_galerkin.real)

### Galerkin Method, Self-Adjoint

In [ ]:
# Bessel equation, self adjoint
L = D(-1)(Nmax,0,ell+1) @ D(-1)(Nmax,0,ell+1).T
M = E(+1)(Nmax,0,ell) @ E(+1)(Nmax,0,ell).T

# Check symmetry
assert np.max(abs(L - L.T)) < 1e-15
assert np.max(abs(M - M.T)) < 1e-15

evalues_symmetric, evectors_symmetric = eigsort(L.todense(), M.todense())
evalues_symmetric = np.sqrt(evalues_symmetric.real)

### Plot the Eigenvalues and Errors

In [ ]:
plt.figure()
plt.plot(evalues_tau.real, evalues_tau.imag, '.', label='tau')
plt.plot(evalues_galerkin.real, evalues_galerkin.imag, '.', label='galerkin')
plt.plot(evalues_symmetric.real, evalues_symmetric.imag, '.', label='symmetric')
plt.legend()
plt.xlabel('Real(λ)')
plt.ylabel('Imag(λ)')
plt.title('Bessel Eigenvalues')
plt.grid()


In [ ]:
def error(vals):
    return np.abs(vals - evalues_analytic[:len(vals)])/np.abs(evalues_analytic[:len(vals)])

errors_tau = error(evalues_tau)
errors_galerkin = error(evalues_galerkin)
errors_symmetric = error(evalues_symmetric)

plt.figure()
plt.semilogy(errors_tau, '.', label='tau')
plt.semilogy(errors_galerkin, '.', label='galerkin')
plt.semilogy(errors_symmetric, '.', label='symmetric')
plt.xlim([0,500])
plt.ylim([10**-16,10**0])
plt.legend()
plt.xlabel('eigenvalue number')
plt.ylabel('relative error')
plt.title('Bessel Eigenvalue Errors')
plt.grid()


In [ ]:
mode_index = 24
mode_coeffs = evectors_tau[:,mode_index]

evalue = evalues_tau[mode_index]
error = errors_tau[mode_index]
print(f'Mode {mode_index}, eigenvalue {evalue:1.5f}, eigenvalue error: {error:1.5e}')

nr = 1000
r = np.linspace(0,1,nr)[1:]
z = 2*r**2 - 1
polys = Zernike.polynomials(dim,Nmax,alpha,ell,z).T
mode = polys @ mode_coeffs

fig, ax = plt.subplots()
ax.plot(r, mode)
ax.set_xlabel('r')
ax.set_ylabel('f')
ax.set_title(f'Bessel Problem, {mode_index}th Eigenmode, l = {ell}')
ax.grid(True)
